In [1]:
%matplotlib inline
from data.dataLoader import *
import torch.utils.data as data
import numpy as np
from models.model_resnet import ResnetDispModel
from Validator import Validator
from trainer import Trainer

In [2]:
val_dataset = TwoViewDataset("data/dataset/", is_train=False, transforms=JointToTensor())
trn_dataset = TwoViewDataset("data/dataset/", is_train=True, transforms=JointToTensor())

In [3]:
val_loader = data.DataLoader(val_dataset, batch_size=1, num_workers=1, shuffle=False)
trn_loader = data.DataLoader(trn_dataset, batch_size=8, num_workers=1, shuffle=False)

In [4]:
network = ResnetDispModel(3)

In [5]:
val = Validator(val_loader, 1, use_gpu=True)

In [6]:
opt = torch.optim.SGD(network.parameters(), lr=1e-2, weight_decay=1e-6,momentum=0.5,
                                nesterov=False)

In [7]:
val.validate(network)

/Users/lizhechen/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
Traceback (most recent call last):
  File "/Users/lizhechen/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/lizhechen/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/lizhechen/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/lizhechen/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [8]:
trn = Trainer(network, trn_loader, opt, 8, use_gpu=True)

ResnetDispModel(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (maxpool): Sequential(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [9]:
val = Validator(network, val_loader,1, use_gpu=False)

In [10]:
val.validate()

/Users/lizhechen/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "


3.1836782455444337